In [25]:
from rdkit import Chem
import math
import pandas as pd
import pickle
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch_geometric.data import Data, Dataset
import pandas as pd
import numpy as np
import seaborn as sns
import pandas_profiling
import joblib
import matplotlib.pyplot as plt
import xgboost as xgb
import json
import pickle
import category_encoders as ce
import openbabel.pybel as pybel
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from utils.utils import *
from utils.surface_area import *
%matplotlib inline

In [5]:
base_path = 'data/test/gridhist'
filenames = os.listdir(base_path)

In [9]:
feat = []
mof_names = []

for idx, filename in enumerate(filenames):
    mof_name = filename.split('.')[0]
    mof_names.append(mof_name)
    
    with open(os.path.join(base_path, filename), "r") as f:
        feat_list = list(map(float, f.read().split(',')))
        feat.append(feat_list)

In [17]:
df_mofname = pd.DataFrame({'MOFname': mof_names})
df_feat = pd.DataFrame(feat)
df = pd.concat([df_mofname, df_feat],axis=1)

In [19]:
df.to_csv('data/test/last_hope_test.csv', index=False)

# Generate Graph

In [26]:
class MOFData(Data):
    def __init__(self, mof_node = None, mof_edge_index = None, mof_edge_attr = None,
                        metal_node = None, metal_edge_index = None, metal_edge_attr = None,
                        organ1_node = None, organ1_edge_index = None, organ1_edge_attr = None,
                        organ2_node = None, organ2_edge_index = None, organ2_edge_attr = None,
                        mofname = None, x_feat = None, y = None):
        super().__init__()
        self.mof_node = mof_node
        self.mof_edge_index = mof_edge_index
        self.mof_edge_attr = mof_edge_attr
        
        self.metal_node = metal_node
        self.metal_edge_index = metal_edge_index
        self.metal_edge_attr = metal_edge_attr
        
        self.organ1_node = organ1_node
        self.organ1_edge_index = organ1_edge_index
        self.organ1_edge_attr = organ1_edge_attr
        
        self.organ2_node = organ2_node
        self.organ2_edge_index = organ2_edge_index
        self.organ2_edge_attr = organ2_edge_attr
        
        self.mofname = mofname
        self.x_feat = x_feat
        self.y = y
        
    def __inc__(self, key, value, *args, **kwargs):
        if key == 'mof_edge_index':
            return self.mof_node.size(0)
        if key == 'metal_edge_index':
            return self.metal_node.size(0)
        if key == 'organ1_edge_index':
            return self.organ1_node.size(0)
        if key == 'organ2_edge_index':
            return self.organ2_node.size(0)
        else:
            return super().__inc__(key, value, *args, **kwargs)

In [27]:
df = pd.read_csv('data/test/clean_test_linker.csv')
gridhist = pd.read_csv('data/test/last_hope_test.csv')
smiles = pd.read_csv('data/test/smiles_test.csv')
data = df.join(smiles.set_index('MOFname'), on='MOFname')
data = data.join(gridhist.set_index('MOFname'), on='MOFname')

data = data.dropna(subset=['Smiles'])
data = data.dropna(subset=['0'])
data = data.reset_index(drop=True)
# data = data.drop('Unnamed: 0', axis=1)
print(data.isnull().sum())
print(data.shape)

MOFname                                          0
topo_0                                           0
topo_1                                           0
topo_2                                           0
topo_3                                           0
topo_4                                           0
topo_5                                           0
topo_6                                           0
topo_7                                           0
topo_8                                           0
topo_9                                           0
volume [A^3]                                     0
weight [u]                                       0
density [g/cm^3]                                 0
surface_area [m^2/g]                             0
void_fraction                                    0
void_volume [cm^3/g]                             0
functional_groups                                0
metal_linker                                     0
organic_linker1                

In [28]:
data_list = []
data_dict = []
drop_feat = ['MOFname', 'weight [u]', 'functional_groups', 'Smiles', 'metal_linker', 'organic_linker1', 'organic_linker2'] #, 'CO2_working_capacity [mL/g]'
c = 1
for _, line in data.iterrows():
    mof = Chem.MolFromSmiles(line['Smiles'])
    metal = Chem.MolFromSmiles(line['metal_linker'])
    organ1 = Chem.MolFromSmiles(line['organic_linker1'])
    organ2 = Chem.MolFromSmiles(line['organic_linker2'])
    
    if mof == None or metal == None or organ1 == None or organ2 == None:  #or metal == None or organ1 == None or organ2 == None
        continue
    
    mof_node, mof_edge_index, mof_edge_attr = generate_graph(mof)
    metal_node, metal_edge_index, metal_edge_attr = generate_graph(metal)
    organ1_node, organ1_edge_index, organ1_edge_attr = generate_graph(organ1)
    organ2_node, organ2_edge_index, organ2_edge_attr = generate_graph(organ2)


    x_feat = line.drop(drop_feat).values.astype(float)
    x_feat = np.expand_dims(x_feat, axis=0)
    x_feat = torch.tensor(x_feat)
    
    data_d = MOFData(mof_node = mof_node, mof_edge_index = mof_edge_index, mof_edge_attr = mof_edge_attr,
                  metal_node = metal_node, metal_edge_index = metal_edge_index, metal_edge_attr = metal_edge_attr,
                  organ1_node = organ1_node, organ1_edge_index = organ1_edge_index, organ1_edge_attr = organ1_edge_attr,
                  organ2_node = organ2_node, organ2_edge_index = organ2_edge_index, organ2_edge_attr = organ2_edge_attr,
                  mofname=line['MOFname'], x_feat=x_feat) #, y=y
#     data_d.mof_num_nodes = len(mof.GetAtoms())
    data_list.append(data_d)
    data_dict.append(line['MOFname'])
    
    if(c%10000==0):
        print('done:',c)
    c=c+1

done: 10000


In [29]:
pickle.dump(data_list, open('data/test/graph_concat_linker_last_hope.pkl', 'wb'))